<font size=4 color=red face='黑体'>4维数组

In [2]:
import numpy as np
x = np.random.rand(10,1,28,28)  #生成10个数据，每个数据的形状为（1,28,28）
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape

(1, 28, 28)

In [4]:
#访问第一个数据的第一个通道的空间数据
x[0,0]

array([[3.63432864e-01, 6.52503544e-01, 6.43321813e-01, 3.31604351e-01,
        8.25106361e-01, 5.63306407e-02, 4.45056781e-01, 4.76298329e-01,
        5.28364553e-02, 7.90381672e-01, 9.34983295e-01, 9.52754012e-01,
        6.27787096e-01, 6.14799367e-01, 4.80650744e-01, 1.84763000e-01,
        4.81712619e-01, 3.14580158e-01, 9.20279126e-01, 7.51396880e-01,
        2.90735485e-01, 2.43292236e-01, 6.15137519e-01, 4.09764076e-01,
        5.97808862e-01, 4.44321323e-01, 6.76665911e-01, 3.65545384e-01],
       [3.25819667e-01, 1.73675500e-01, 7.46450938e-01, 2.56943854e-02,
        5.64064205e-01, 6.53597859e-02, 5.82177378e-01, 3.15878450e-01,
        9.07952078e-01, 4.83139399e-01, 6.11471044e-01, 2.85470254e-01,
        3.72182894e-01, 5.27854290e-01, 2.65869593e-02, 7.00046724e-01,
        7.49932661e-01, 8.42906821e-01, 3.80925481e-01, 7.93140262e-01,
        8.97365738e-01, 1.59303614e-01, 2.43288165e-01, 9.68417812e-01,
        2.97129677e-01, 7.53532417e-02, 1.66809852e-01, 1.82425

<font size=4 color=red face='黑体'>定义softmax函数

In [9]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)  #防止溢出
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

<font size=4 color=red face='黑体'>定义cross_entropy_error函数

In [10]:
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
        
    batch_size = y.shape[0]
    if t.size == y.size:
        error = -np.sum(t * np.log(y + 1e-7)) / batch_size
    else:
        error = -np.sum(np.log(y[np.arange(batch_size),t] + 1e-7)) / batch_size
        
    return error

<font size=4 color=red face='黑体'>回归问题使用恒等函数，分类问题使用softmax函数，softmax函数对应使用交叉熵误差计算损失函数，恒等函数对应使用平方和误差计算损失函数，这样在反向传播时可以得到（yi-ti）这样的结果。

<font size=4 color=red face='黑体'>定义ReLU层的类：Relu

In [11]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self,x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0 #将小于0的数转化为0
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        return dx

<font size=4 color=red face='黑体'>定义Sigmoid层的类：Sigmoid

In [12]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self,x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self,dout):
        dx = dout * (1 - self.out) * self.out
        return dx

<font size=4 color=red face='黑体'>定义Affine层的类：Affine

In [13]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        
        self.dW = None
        self.db = None
        
    def forward(self,x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0],-1)
        self.x = x
        
        out = np.dot(self.x,self.W) + self.b
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        
        dx = dx.reshape(*self.original_x_shape) #还原输入数据的形状
        return dx

<font size=4 color=red face='黑体'>定义Softmax-with-Loss层的类：SoftmaxWithLoss

In [14]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None  #softmax的输出
        self.t = None  #监督数据
        
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:   #监督数据是one-hot-vector的情况
            dx = (self.y - self.t)/batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size),self.t] -= 1
            dx = dx/batch_size
            
        return dx

<font size=3 face='黑体'>softmax函数和cross_entropy_error函数见上

In [6]:
#im2col函数：将3维或4维数据转换为2维矩阵
import numpy as np
from common.util import im2col
from common.util import col2im

In [2]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)

(9, 75)


<font size=4 color=red face='黑体'>定义卷积运算的类：Convolution

In [7]:
class Convolution:
    def __init__(self,W,b,stride=1,pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.col = None
        self.col_W = None
        
        self.dW = None
        self.db = None
        
    def forward(self,x):
        FN,C,FH,FW = self.W.shape
        N,C,H,W = x.shape
        out_h = int(1+(H + 2*self.pad - FH)/self.stride)
        out_w = int(1+(W + 2*self.pad - FW)/self.stride)
        
        col = im2col(x,FH,FW,self.stride,self.pad)
        col_W = self.W.reshape(FN,-1).T
        
        out = np.dot(col,col_W)+self.b
        out = out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)  #transpose：改变多维数组轴的顺序
        
        self.x = x
        self.col = col
        self.col_W = col_W
        
        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
        return dx

<font size=4 color=red face='黑体'>backward中只有Affine层和Convolution层计算参数的梯度，其他的层（包括Affine和Convolution）进行数据的传递

<font size=4 color=red face='黑体'>定义池化的类：Pooling

In [15]:
class Pooling:
    def __init__(self,pool_h,pool_w,stride=1,pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None
        
    def forward(self,x):
        N,C,H,W = x.shape
        out_h = int(1+(H-self.pool_h)/self.stride)
        out_w = int(1+(W-self.pool_w)/self.stride)
        
        #展开
        col = im2col(x,self.pool_h,self.pool_w,self.stride,self.pad)
        col = col.reshape(-1,self.pool_h * self.pool_w)
        
        #取最大值
        out = np.max(col,axis=1)
        
        #转换
        out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)
        
        arg_max = np.argmax(col,axis=1)
        self.x = x
        self.arg_max = arg_max
        
        return out
    
    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx

<font size=4 color=red face='黑体'>CNN的实现(创建一个简单的卷积神经网络（上面各个层的拼接）来学习手写数字图像识别)

In [1]:
from collections import OrderedDict
import pickle
from common.layers import *
import numpy as np
from common.util import im2col
from common.util import col2im

class SimpleConvNet:
    #初始化部分
    def __init__(self,input_dim=(1,28,28),
                 conv_param={'filter_num':30,'filter_size':5,'pad':0,'stride':1},
                hidden_size=100,output_size=10,weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad)/filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))  
        #因为下文设定的Pooling层的size=2，stride=2，把Pooling层的输出展开为一行，才可以进行下面的Affine层
        
        #参数的初始化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num,input_dim[0],filter_size,filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size,hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b3'] = np.zeros(output_size)
        
        #生成必要的层
        self.layers = OrderedDict()   #生成有序字典
        self.layers['Conv1'] = Convolution(self.params['W1'],self.params['b1'],conv_param['stride'],conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2,pool_w=2,stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'],self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'],self.params['b3'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    #进行预测与计算损失函数
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]
    
    #最后基于误差反向传播法求梯度
    def gradient(self,x,t):
        #forward
        self.loss(x,t)
        
        #backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        #设定
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads
    
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

<font size=4 color=red face='黑体'>使用这个SimpleConvNet来学习MNIST数据集

In [2]:
import pickle
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer
from common.optimizer import *

# 读入数据
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 处理花费时间较长的情况下减少数据 
x_train, t_train = x_train[:5000], t_train[:5000]
x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 保存参数
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 绘制图形
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.2995897386427537
=== epoch:1, train acc:0.286, test acc:0.298 ===
train loss:2.298181725397648
train loss:2.2926647930895343
train loss:2.2891189497689965
train loss:2.281954914680036
train loss:2.2708708539408873
train loss:2.247008451106033
train loss:2.2458318689962598
train loss:2.214124017944188
train loss:2.1884758260231805
train loss:2.146348235498979
train loss:2.1076805738665154
train loss:2.094495635157899
train loss:2.0290312653670357
train loss:2.0821116903678534
train loss:1.9489001707835039
train loss:1.9055595840417832
train loss:1.838576732567927
train loss:1.7355329508718733
train loss:1.5971692878242454
train loss:1.594581318542135
train loss:1.4742619381475703
train loss:1.3398282705972082
train loss:1.3389788684273944
train loss:1.245954355632327
train loss:1.2253547110316914
train loss:1.0605415218756171
train loss:0.8998996379568663
train loss:0.8864097897525731
train loss:0.8044214645190374
train loss:0.9486363317659784
train loss:0.7415725089518904


train loss:0.17170562375427376
train loss:0.2389976074171121
train loss:0.3026042088713802
train loss:0.11340398251540011
train loss:0.24459626510871194
train loss:0.25712450405680387
train loss:0.15101039063447902
train loss:0.2678002135395573
train loss:0.14582667259099982
train loss:0.128074275137873
train loss:0.18634401549531313
train loss:0.13395405528142437
train loss:0.19060367677427734
train loss:0.26214667460540514
train loss:0.27826775364910383
train loss:0.2617081321460271
train loss:0.17326423969673155
train loss:0.1532724953082087
train loss:0.10753226728463802
train loss:0.12681194873275733
train loss:0.08959188133121918
train loss:0.09698300680427333
train loss:0.16933774818286945
train loss:0.18194631136718814
train loss:0.19548434990688546
train loss:0.21894361933459183
train loss:0.11924612860760055
train loss:0.28874671901477417
train loss:0.18686017068957878
train loss:0.18291792572778331
train loss:0.18163674285037293
train loss:0.1984343650020705
train loss:0.142

train loss:0.03690786888192541
train loss:0.09000000972048246
train loss:0.10432401190260648
train loss:0.07819698205926646
train loss:0.06907932990750332
train loss:0.08631141030378674
train loss:0.07898263064390904
train loss:0.03547872524233215
train loss:0.10673937136381158
train loss:0.11395283399847585
train loss:0.09326101547464172
train loss:0.05018038535533121
train loss:0.03833308832355962
train loss:0.10208973443231568
train loss:0.05488701369657993
train loss:0.07124380333644866
train loss:0.04707552275781133
train loss:0.07553510570184337
train loss:0.06473427500835544
train loss:0.2242698668692929
train loss:0.09390335708464839
train loss:0.06373264944381009
train loss:0.05985710017282558
train loss:0.06670265383470188
train loss:0.0437268507647047
train loss:0.10103053006623157
train loss:0.06307471210713143
train loss:0.15078438695563692
train loss:0.06377892638521955
train loss:0.07833249489538162
train loss:0.07602370688756617
train loss:0.03902175931934234
=== epoch:

train loss:0.04266544589369901
train loss:0.020517719318000532
train loss:0.030295013148491347
train loss:0.031803257235390285
train loss:0.018954484786708987
train loss:0.0493304480871366
train loss:0.029345113752565433
train loss:0.026574189781847873
train loss:0.021287891819535165
train loss:0.0239422107116167
train loss:0.04950704871029379
train loss:0.030181660037827625
train loss:0.021395743922893652
train loss:0.07669206659740677
train loss:0.12310324657231538
train loss:0.0242658509614248
train loss:0.031526640516072596
train loss:0.03152337392764629
train loss:0.046668939928866456
train loss:0.03381496124022909
train loss:0.03646658798582071
train loss:0.026414517317157937
train loss:0.04443043234783775
train loss:0.01901654751506737
train loss:0.025922224796286898
train loss:0.011692508948007751
=== epoch:17, train acc:0.99, test acc:0.954 ===
train loss:0.029078750188832925
train loss:0.023478714261374493
train loss:0.039196657643961734
train loss:0.04293139647844635
train l

<Figure size 640x480 with 1 Axes>